In [1]:
import UWGeodynamics as GEO
import numpy as np
from UWGeodynamics import visualisation as vis
from underworld import function as fn
import underworld as uw
import os
from underworld import mesh

loaded rc file /opt/venv/lib/python3.9/site-packages/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
u = GEO.UnitRegistry

half_rate = 1.0 * u.centimeter / u.year
model_length = 6000. * u.kilometer
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1900. * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM


Model = GEO.Model(elementRes=(34, 34, 12),
                  minCoord=(0. * u.kilometer, -0. * u.kilometer, -680. * u.kilometer),
                  maxCoord=(4000. * u.kilometer, 2000. * u.kilometer, 20. * u.kilometer),
                  gravity=(0.0 ,0.0, -9.81 * u.meter / u.second**2))

Model.outputDir="tiptracers"

StickyAir = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer3D(top=Model.top, bottom=0. * u.kilometer))

OverPlate = Model.add_material(name="OverPlate", shape=GEO.shapes.Layer3D(top=0. * u.kilometer, bottom=-120. * u.kilometer))

Mantle = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=-120. * u.kilometer, bottom=Model.bottom))

top    = GEO.shapes.HalfSpace(normal=(0.,0.,1.),
            origin=(6450.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
right  = GEO.shapes.HalfSpace(normal=(1.,0.,0.),
            origin=(9000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
bottom = GEO.shapes.HalfSpace(normal=(0.,0.,-1.),
            origin=(6450.*u.kilometer,1000.*u.kilometer,-120.*u.kilometer))
left   = GEO.shapes.HalfSpace(normal=(-1.,0.,1.),
            origin=(1400.*u.kilometer,1000.*u.kilometer,-60.*u.kilometer))
back = GEO.shapes.HalfSpace(normal=(0.,1.,0.),
            origin=(6450.*u.kilometer,1750.*u.kilometer,-60.*u.kilometer))
front = GEO.shapes.HalfSpace(normal=(0.,-1.,0.),
            origin=(6450.*u.kilometer,250.*u.kilometer,-60.*u.kilometer))
CompositeShape_Polygon1 = top & right & bottom & left & back & front

top2    = GEO.shapes.HalfSpace(normal=(0.,0.,1.),
            origin=(1500.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))

right2  = GEO.shapes.HalfSpace(normal=(1.,0.,-1.),
            origin=(1437*u.kilometer,1000.*u.kilometer,-168.*u.kilometer))

bottom2 = GEO.shapes.HalfSpace(normal=(-1.,0.,0.),
            origin=(1280*u.kilometer,1000.*u.kilometer,-350.*u.kilometer))


left2   = GEO.shapes.HalfSpace(normal=(-1.,0.,1.),
            origin=(1320.*u.kilometer,1000.*u.kilometer,-112.*u.kilometer))

back = GEO.shapes.HalfSpace(normal=(0.,1.,0.),
                            origin=(1467.*u.kilometer,1750.*u.kilometer,-1436.*u.kilometer))
front = GEO.shapes.HalfSpace(normal=(0.,-1.,0.),
                            origin=(1467.*u.kilometer,250.*u.kilometer,-146.*u.kilometer))

CompositeShape_Polygon2 = top2 & right2 & bottom2 & left2 & back & front

CompositeShape = CompositeShape_Polygon1 | CompositeShape_Polygon2
Lithosphere = Model.add_material(name="Lithosphere", shape=CompositeShape)

OverPlate.viscosity = 1e20 * u.pascal * u.second
StickyAir.viscosity = 5e18 * u.pascal * u.second
Lithosphere.viscosity = 1e24 * u.pascal * u.second 

rh = GEO.ViscousCreepRegistry()
#dir(rh)
Mantle.viscosity = rh.Dry_Olivine_Dislocation_Korenaga_and_Karato_2008

OverPlate.density = 3180. * u.kilogram / u.metre**3
StickyAir.density = 1. * u.kilogram / u.metre**3
Mantle.density = 3200. * u.kilogram / u.metre**3
Lithosphere.density = 3340. * u.kilogram / u.metre**3

pl = GEO.PlasticityRegistry()
#dir(pl)
OverPlate.plasticity = pl.Rey_and_Muller_2010_LithosphericMantle
Lithosphere.plasticity = pl.Rey_and_Muller_2010_LithosphericMantle
Mantle.plasticity = pl.Watremez_et_al_2013_AsthenosphericMantle

#Lithosphere.elasticity = GEO.Elasticity(shear_modulus=4.*u.gigapascal, observation_time=20000. * u.year)

# Boonma bibliography, escull aquesta perquè li convé
#Model.diffusivity = 1e-06 * u.metre**2 * u.seconds**-1   
#Mantle.diffusivity = 1e-05 * u.metre**2 * u.seconds**-1  

# Meva pròpia
Model.diffusivity = 1e-06 * u.metre**2 * u.seconds**-1
OverPlate.diffusivity = 1e-06 * u.metre**2 * u.seconds**-1 
Mantle.diffusivity = 2e-05 * u.metre**2 * u.seconds**-1 
Lithosphere.diffusivity = 1e-06 * u.metre**2 * u.seconds**-1 
StickyAir.diffusivity = 2e-05 * u.metre**2 * u.seconds**-1    # http://www.mhtl.uwaterloo.ca/old/onlinetools/airprop/airprop.html

Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)
Mantle.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)


Model.set_velocityBCs(left=[0., None, None], right=[0., None, None],
                      bottom=[None, None, 0.], top=[None, None, 0.],
                      front=[None, 0., None], back=[None, 0., None])

Model.set_temperatureBCs(top=surfaceTemp, bottom=baseModelTemp)


strainRateTensor = fn.tensor.symmetric(Model.velocityField.fn_gradient)

secondInvariantStrainRateTensor = fn.tensor.second_invariant(
                      fn.tensor.deviatoric(
                      fn.tensor.symmetric(Model.velocityField.fn_gradient)))

velocityGradient = Model.velocityField.fn_gradient

cxpos = np.linspace(GEO.nd(1400. * u.kilometer), GEO.nd( 2000. *u.kilometer), 10)
cypos = np.linspace(GEO.nd(300. * u.kilometer), GEO.nd(1700. * u.kilometer), 10)
czpos = np.linspace(GEO.nd(-150. * u.kilometer), GEO.nd(-600. * u.kilometer), 10)
#z 
cxpos, cypos, czpos = np.meshgrid(cxpos, cypos, czpos)
centroid_coords = np.ndarray((len(cxpos.ravel()), 3))

centroid_coords[:, 0] = cxpos.ravel()
centroid_coords[:, 1] = cypos.ravel()
centroid_coords[:, 2] = czpos.ravel()

point = np.zeros((1, 3))

Model.add_passive_tracers(name="lag", vertices=point, centroids=centroid_coords)



	Global element size: 34x34x12
	Local offset of rank 0: 0x0x0
	Local range of rank 0: 34x34x12
In func WeightsCalculator_CalculateAll(): for swarm "ENIXI2YM__swarm"
	done 33% (4624 cells)...
	done 67% (9248 cells)...
	done 100% (13872 cells)...
WeightsCalculator_CalculateAll(): finished update of weights for swarm "ENIXI2YM__swarm"


In [5]:
x1, y1, z1 = GEO.nd(1600. * u.km), GEO.nd(455.555556 * u.km), GEO.nd(-400. * u.km)
x2, y2, z2 = GEO.nd(1600. * u.km), GEO.nd(1544.444444 * u.km), GEO.nd(-400. * u.km)

#455
x3, y3, z3 = GEO.nd(1733.333333 * u.km), GEO.nd(922.222222 * u.km), GEO.nd(-400. * u.km)


coords = np.ndarray((3, 3))
coords[:, 0] = np.array([x1, x2, x3])
coords[:, 1] = np.array([y1, y2, y3])
coords[:, 2] = np.array([z1, z2, z3])

Model.add_passive_tracers(name="tip", vertices=coords)

In [ ]:
df.iloc[835]
0    1733.333333
1     922.222222
2    -400.000000

In [ ]:
Model.tip_tracers.add_tracked_field(velocityGradient,
                                           name="velocityGradient",
                                           units=u.second**-1,
                                            dataType="float",
                                           count=9,
                                            overwrite=True)


Model.tip_tracers.add_tracked_field(Model.velocityField,
                                           name="velocityField",
                                           units=u.m / u.second,
                                            dataType="float",
                                           count=3,
                                            overwrite=True)

Model.tip_tracers.add_tracked_field(strainRateTensor,
                              name="strainRateTensor",
                              units= u.second**-1,
                              dataType="float",
                              count=6,
                              overwrite=True)


Model.init_model(temperature="steady-state", pressure="lithostatic")



Model.solver.set_inner_method("mumps")
Model.solver.set_penalty(1e6)
GEO.rcParams['nonlinear.tolerance'] = 2.9e-2
GEO.rcParams['initial.nonlinear.tolerance'] = 2.9e-2


In [ ]:

Model.run_for(100. * u.megayears, checkpoint_interval=.5* u.megayears)